In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pickle

from sklearn.metrics import jaccard_similarity_score
from scipy.stats import pearsonr

In [7]:
bookmark_url = 'https://www.yelp.com/user_details_bookmarks?userid='
review_url = 'https://www.yelp.com/user_details_reviews_self?userid='
friends_url = 'https://www.yelp.com/user_details_friends?userid='
self_id = 'ShHBKjuJbQAVBLs7DgA95A'

In [8]:
def first_page_friends(user_id):
    """function to get the User IDs of the first page of friends"""

    f_page = requests.get(friends_url + user_id)
    f_soup = BeautifulSoup(f_page.content, 'html.parser')
    f_source = f_soup.find_all('a', class_='user-display-name js-analytics-click')
    
    friend_list = []
    for friend in range(0, len(f_source)):
        friend_id = f_source[friend]['href'].split('=')[1]
        friend_list.append(friend_id)
    
    return friend_list

In [9]:
def get_all_friends(user_id):
    """function to get the User IDs of all friends"""
    
    # get the max page number of bookmarks
    friendpage = requests.get(friends_url+user_id)
    fr_soup = BeautifulSoup(friendpage.content, 'html.parser')
    pages = fr_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
    max_page = pages[0].get_text().split('of ')[1].split('\n')[0]
    friend_list = []   
    #get friends
    for page_num in range(0,48*int(max_page), 48):
        f_page = requests.get(friends_url + user_id+ '&start=' + str(page_num))
        f_soup = BeautifulSoup(f_page.content, 'html.parser')
        f_source = f_soup.find_all('a', class_='user-display-name js-analytics-click')
    
        for friend in range(0, len(f_source)):
            friend_id = f_source[friend]['href'].split('=')[1]
            friend_list.append(friend_id)

    return friend_list

In [10]:
def get_bookmarks(user_id):
    """get all bookmarks for user_id"""

    user_bookmarks = []
     
    try: # not all accounts have the bookmarks as public data       
   
        # get the max page number of bookmarks
        bookmarks = requests.get(bookmark_url+user_id)
        bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
        pages = bm_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
        max_page = pages[0].get_text().split('of ')[1].split('\n')[0]

        # loop through all the pages to get bookmarks
        for page_num in range(0, 50*int(max_page), 50):
            bookmarks = requests.get(bookmark_url + user_id + '&start=' + str(page_num))
            bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
            bm_name = bm_soup.find_all('a', class_='biz-name js-analytics-click')

            for mark in range(0, len(bm_name)):
                user_bookmarks.append(bm_name[mark]['href'])
        
        return user_bookmarks
    
    except: 
        return [np.nan]
       


In [11]:
def get_bookmarks(user_id):
    """get all bookmarks for user_id"""

    user_bookmarks = []
     
    try: # not all accounts have the bookmarks as public data       
   
        # get the max page number of bookmarks
        bookmarks = requests.get(bookmark_url+user_id)
        bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
        pages = bm_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
        max_page = pages[0].get_text().split('of ')[1].split('\n')[0]

        # loop through all the pages to get bookmarks
        for page_num in range(0, 50*int(max_page), 50):
            bookmarks = requests.get(bookmark_url + user_id + '&start=' + str(page_num))
            bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
            bm_name = bm_soup.find_all('a', class_='biz-name js-analytics-click')

            for mark in range(0, len(bm_name)):
                user_bookmarks.append(bm_name[mark]['href'])
        
        return user_bookmarks
    
    except: 
        return [np.nan]
       



In [473]:
def biz_id_bookmarks(user_id):
    """get all bookmarks for user_id"""

    bizid_bookmarks = []
     
    try: # not all accounts have the bookmarks as public data       
   
        # get the max page number of bookmarks
        bookmarks = requests.get(bookmark_url+user_id)
        bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
        pages = bm_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
        max_page = pages[0].get_text().split('of ')[1].split('\n')[0]

        # loop through all the pages to get bookmarks
        for page_num in range(0, 50*int(max_page), 50):
            bookmarks = requests.get(bookmark_url + user_id + '&start=' + str(page_num))
            bm_soup = BeautifulSoup(bookmarks.content, 'html.parser')
            biz_id = bm_soup.find_all('a', class_='biz-name js-analytics-click')

            for mark in range(0, len(biz_id)):
                bizid_bookmarks.append(biz_id[mark]['data-hovercard-id'])
           
    except: 
        bizid_bookmarks.append(np.nan)

    return bizid_bookmarks

    

In [500]:
def biz_id_reviews(user_id):
        
    r_biz_id = []
    
    try: # not all accounts have the reviews    
        
        # get the max page number of reviews
        reviews = requests.get(review_url+user_id)
        re_soup = BeautifulSoup(reviews.content, 'html.parser')
        pages = re_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
        max_page = pages[0].get_text().split('of ')[1].split('\n')[0]
        
        # loop through all the pages to get reviews   
        for page_num in range(0, 10*int(max_page), 10):
            rev = requests.get(review_url + user_id + '&rec_pagestart=' + str(page_num))
            rev_soup = BeautifulSoup(rev.content, 'html.parser')
            #rating = rev_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
            biz_id = rev_soup.find_all('a', class_='biz-name js-analytics-click')

            for mark in range(0, len(biz_id)):
                r_biz_id.append(biz_id[mark]['data-hovercard-id'])

    except:
        r_biz_id.append(np.nan)
    
    return r_biz_id   

In [19]:
def get_reviews(user_id):
    """get all reviews for user_id"""
    
    try:
        # get the max page number of bookmarks
        reviews = requests.get(review_url+user_id)
        re_soup = BeautifulSoup(reviews.content, 'html.parser')
        pages = re_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
        max_page = pages[0].get_text().split('of ')[1].split('\n')[0]

        user_reviews = {}
        # loop through all the pages to get bookmarks
        for page_num in range(0, 10*int(max_page), 10):
            rev = requests.get(review_url + user_id + '&rec_pagestart=' + str(page_num))
            rev_soup = BeautifulSoup(rev.content, 'html.parser')
            rating = rev_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
            biz_name = rev_soup.find_all('a', class_='biz-name js-analytics-click')

            for mark in range(0, len(biz_name)):
                user_reviews[biz_name[mark]['href']] = int(rating[mark]['title'][0])

    
    except:
        pass # pass for users with no reviews
    
    return user_reviews


In [15]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [472]:
myrev_bizid = biz_id_reviews(self_id)
myrev_bizid

['v2aPDoERIn_doAXpyqGYOw',
 'NebnYfJWMtsKFZSkvDPEsA',
 'ceUQhn_QlqCc6m__brQmzA',
 '_cdPsUIowDUJ7epysm_X6A',
 'ppOhdWOkjDPCegKuaW2aVQ',
 'KgpYf7igA74ExSIVq-a4zw',
 '4jCR2shGR3GaKCL1XS1eBQ',
 'Etm5xTNkA6zQY2Tz9_YUeQ',
 'T_oMJKCn3oZ2EsZcrk-NPg',
 '8N8Jh1EMN50m3smF-aLRnw',
 'eCDdnpkKl8joLLQsT8bVZw',
 'jHtYlqQw0BBV2bm6uGKCfg',
 'ret2Mb3O7FOcJySLlL5dZQ',
 '673JaQLFNEn23uNz7dnSTA',
 'b8xBuAGRVsgiWP5bRgegVg',
 '2HMr99Ouj2PzyhVLn3iBfw',
 'gQ4np2dHCwgXdeFX-cPDmA',
 'QE9zhEDWhF_VM-jeOgGfrg',
 '1x3nLsNu3vboe7q06eUhRw',
 'AB3ma0ghyHuJjlKyI4iKSg',
 'ROhNE6fqKIy3YrLSMDhwCg',
 'mhfbd0KSf8Bny2UCedgfzg',
 '1DI-Giw_bCIWSXmx3pdi5A',
 'RzGcmx04nt2cL3upgZQ7ig',
 'wd4BNecY22NqqFLGHoKDaw',
 'BJEGI7IEreRHShCitewZEQ',
 'faaWrHESothOI01gPYeZ_Q',
 '_g8AIAoj2Rz1pT-B1pq7Sg',
 '4hPeixyObMTNE4B4GCNTVA',
 'FY5GNGTUo5C8DZpGOzgTCA',
 'rk-Bv5Qpr-qH8HOE9ecrxA',
 'bRSAIuyNZ-CoWFzPmmRk5w',
 'U4ZOtih0A1qTfg2aMlcLXQ',
 'AZv5MtRCkanN2eauJ0KqDQ',
 'fuBcibqLBhyPp95Q1yhxwQ',
 'x66vf8mt51PAwVHFiIBf1w',
 '3qWPi4sffCmcVtysdWMzPg',
 

In [501]:
# need to rerun
friendrev_bizid = {}

for user in all_friends:
    friendrev_bizid[user] = biz_id_reviews(user)

In [502]:
friendrev_bizid

{'-OKmukwdCrHq6bkF2_gSwQ': [nan],
 '-uo4C6bEp5Ideik30QRMjQ': [nan],
 '1LFfY-EOeuoYhoTiNTGgyw': [nan],
 '1bIhzyBSK5wY05z0IFMBow': [nan],
 '2FDgACZM1XlzyjGY19imjw': [nan],
 '3ACKvpVPFMvopThjomGjfA': [nan],
 '3OqUfYNRNYewg-1M9gdAvQ': [nan],
 '3d6EmL_kSvoqAt9Si5CZgg': [nan],
 '5RifcJP_Lf-MzojTHybBNw': [nan],
 '5wKI0_5JerDnfQ3muIOCjQ': [nan],
 '6WSRbCjD0-p39qxobiBIzA': [nan],
 '6sDNWHF_jKE9rIjbc5ZfOw': [nan],
 '7IV3JXTXufasWNSkDEdrcQ': [nan],
 '7XdR2OvFXBoOmQ05pALzmg': [nan],
 '8yd7WNNbkdgSF_b6yNHv0w': [nan],
 '9K7YBhT8PHka8QaaSCiuuw': [nan],
 'AaB2ate8xPjdhIy59VCSpw': [nan],
 'AtBlOFl4FUtmLOzebCibEw': [nan],
 'BUtOFEDzoytIFbilVc2Fcw': [nan],
 'Bvp2Kx0M-GrcUOg1w0UGsw': [nan],
 'C7wCMDIk0v6dLQX5ywFvRg': [nan],
 'D7O2Euol8WGTGFWF6klNqg': [nan],
 'DBGOwUcsAKZKaBcTMbgG8w': [nan],
 'DGg7tGXi9McTdnZjdps34w': [nan],
 'DouEJ-59eB6QJFuxQ9kYEA': [nan],
 'E87rG1L3DAeOL5Xu-U5TKA': [nan],
 'EGUhiD-EmzMklDHp3YmNsQ': [nan],
 'F4rYrFoXCrvBoM7_0XOKsQ': [nan],
 'FOdWCgLWE0b7QMcu3Q9MYQ': [nan],
 'FraRROMnu9ro

In [478]:
mybm_bizid = biz_id_bookmarks(self_id)
mybm_bizid

['CebK2lhTzSz8SLIocEwLTg',
 'O1WxBWrQnq-OsZDTabdVlw',
 'E0q1KbNzbJd991Hh5zUg2Q',
 '92BIaREj1SJWJKL3sbBNrg',
 'QAxRjaiiIYFI_6HjbR8ltw',
 'dqk3C4Uo0inycVUFa5K-Sg',
 'QEbdR0f3rUl0njt1oVGWnw',
 'Lib4Z4cwDdmjadEHVQGPiA',
 'DVaTHt3XOOrFhsbY24T5qA',
 'c-ZLKbVOZ28pEw9_gUPDOA',
 'xa3du4kkt0tpbA3NueM9lg',
 'nqI4EL3RCi-ksL5QpBHJig',
 'lucM323BHQ7rGsVqgXa_5A',
 'Sil6pqwwBLarXEFu1BAMJg',
 'onk3_CJgjvjRf5cfVtC7vA',
 'Jpcv2ipU_4AbIx-SOnzzFA',
 'fBZ65r2C1gIiCM_WWzEt8g',
 'Hvve0-Qr5tywdacj734akw',
 'wuvjUpFeF2iH1T-G9FbB_g',
 '_0oNHdVXtH0nOzIIQdooXw',
 'PCs4R-Ut6KDGbH6ApRiTBA',
 'cMfHQGklDreBwRIvh3DOgQ',
 'IEWLxfMf6393HsHkQbxHmg',
 '_ToAzDXwTYx6VVrnhBWisg',
 'OAar-j89d4zJxUM48mi4rA',
 'QocZr0cm1CjNHzJgLVtp1g',
 '_xC5B2GytD5xOJZ5l3hM1Q',
 'h36Q3hnWTiVP5cpWuRfhFA',
 '8a1eKKAS3b2KdYHnAo63eg',
 '8vXqLyMm8gWQbOt31IEafQ',
 'SeHGCBje7ANj1lSK_5j9uQ',
 '8Bby-miG85cKgeCMp8QGhA',
 'gX-9nqcdJLQHXuXVFIKMzw',
 'CPRzsH7GLM9NSyDuW5ieOQ',
 'ztCVIrq0zqgBN688VsVomQ',
 '_aKMnIO-9S8SQTcf3yWoUA',
 '9dDtdToNlsNxA7F8s6Ek9g',
 

In [520]:
%store myrev_bizid
%store friendrev_bizid
%store mybm_bizid

Stored 'myrev_bizid' (list)
Stored 'friendrev_bizid' (dict)
Stored 'mybm_bizid' (list)


In [519]:
print('My bookmarks : {}'.format(len(mybm_bizid)))
print('My reviews : {}'.format(len(myrev_bizid)))

My bookmarks : 249
My reviews : 67


In [479]:
friendbm_bizid = {}

for user in all_friends:
    friendbm_bizid[user] = biz_id_bookmarks(user)

In [ ]:
# delete this 
def get_reviews_2(user_id):
    """get all reviews for user_id"""
    
    # get the max page number of reviews
    reviews = requests.get(review_url+user_id)
    re_soup = BeautifulSoup(reviews.content, 'html.parser')
    pages = re_soup.find_all('div', class_='page-of-pages arrange_unit arrange_unit--fill')
    max_page = pages[0].get_text().split('of ')[1].split('\n')[0]
    
    user_reviews = {}
    # loop through all the pages to get reviews
    for page_num in range(0, 10*int(max_page), 10):
        rev = requests.get(review_url + user_id + '&rec_pagestart=' + str(page_num))
        rev_soup = BeautifulSoup(rev.content, 'html.parser')
        rating = rev_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
        biz_name = rev_soup.find_all('a', class_='biz-name js-analytics-click')
        cuisine = rev_soup.find_all('span', class_='category-str-list')
        
        for mark in range(0, len(biz_name)):
            user_reviews[biz_name[mark]['href']] = []
            user_reviews[biz_name[mark]['href']].append(int(rating[mark]['title'][0]))
       
    return user_reviews

In [511]:
# test
reviews = requests.get(review_url+all_friends[1])
re_soup = BeautifulSoup(reviews.content, 'html.parser')
pages = re_soup.find_all('div', class_="page-of-pages arrange_unit arrange_unit--fill")
max_page = pages#[0].get_text().split('of ')[1].split('\n')[0]

#rev = requests.get(review_url + user_id + '&rec_pagestart=' + str(page_num))
#rev_soup = BeautifulSoup(rev.content, 'html.parser')


In [469]:
#rating = rev_soup.find_all('div', class_=re.compile('i-stars i-stars--regular-*'))
#biz_name = rev_soup.find_all('a', class_='biz-name js-analytics-click')
biz_id = rev_soup.find_all('a', class_='biz-name js-analytics-click')

test = []
for mark in range(0, len(biz_id)):
    test.append(biz_id[mark]['data-hovercard-id'])
test


10

In [290]:
def normalize(row):
    mean = row.sum() / row.count()
    row = row - mean
    return row

In [466]:
client_id = '3ywXGq4PyUoR_XXDvK_RIA'
api_key = 'FPOb0CFdwPxb8_4qN-r89V7n_74JWBe-xwT2zMhRXpd0oCxsn22aY0aITmZZo7FGZvzeKyYMlu7P4C72DXyEAxqpB3_A9aPRjJH1RBXkkwnsoilWCjJSwm-N-Vj2W3Yx'

# Purpose: 
#### The purpose is to build a recommendation system for Yelp using bookmarks and review ratings

To build a Yelp recommender engine, I will be looking at a Yelp Elite account. The data will be scraped from www.Yelp.com and used to create a recommendation engine based on collaborative filtering. I will scrape the list of friends and for each friend, get the list of their bookmarks and their review ratings. Once that is obtained, I can compare the users to find the ones that are most similar to my account and look at those items to generate a set of recommendations

## Getting the data - Bookmarks

In [3]:
# restore all stored variables
%store -r

First, I will need to obtain a list of my friend's user IDs. Instead of retreiving all friends (total of 144), I will only be retrieving 48 (only the first page) so that my processing times will be cut down. 

I also determined that the first page of friends should be sufficient as it appears that Yelp's algorithm doesn't order friends by alphabetical order but rather will sort all your Yelp Elite friends on the first page. Since that appears to be the case, I am more comfortable with only using the first page of friends since in order to be Yelp Elite, you must have many reviews.

Note: if you'd like to recreate this recommendation but with using all friends, you can use the function `get_all_friends()` instead of `first_page_friends()`. 

In [292]:
# run function to get first page of friends
source_friends = first_page_friends(self_id)

In [293]:
%store source_friends

Stored 'source_friends' (list)


In [294]:
len(source_friends)

48

In [417]:
all_friends = get_all_friends(self_id)

In [418]:
%store all_friends

Stored 'all_friends' (list)


In [419]:
len(all_friends)

144

Once I have a list of my friend's user IDs, I will need to obtain their lists of  bookmarks and my own list of bookmarks.

In [6]:
save_obj(all_friends, 'all_friends')

In [8]:
save_obj(self_bookmarks, 'self_bookmarks')
save_obj(all_bookmarks, 'friend_bookmarks')
save_obj(all_reviews, 'friends_reviews')
save_obj(all_reviews_vector, 'all_reviews_vector')

In [295]:
# list for my own bookmarks
self_bookmarks = get_bookmarks(self_id)
self_bookmarks

['/biz/three-twins-ice-cream-san-francisco-5',
 '/biz/ben-thai-cafe-san-francisco',
 '/biz/ijji-sushi-san-francisco',
 '/biz/koi-palace-daly-city',
 '/biz/robin-san-francisco',
 '/biz/maria-catita-restaurante-e-loja-regional-lisboa',
 '/biz/juns-beauty-salon-san-francisco',
 '/biz/bowld-acai-san-francisco-2',
 '/biz/china-bee-san-mateo',
 '/biz/bobos-san-francisco-14',
 '/biz/grand-lake-kitchen-oakland',
 '/biz/nabe-san-francisco-5',
 '/biz/dumpling-kitchen-san-francisco',
 '/biz/nojo-ramen-tavern-san-francisco-2',
 '/biz/el-milagro-los-angeles-2',
 '/biz/five-happiness-san-francisco',
 '/biz/tadich-grill-san-francisco',
 '/biz/mahalo-bowl-sunnyvale-2',
 '/biz/hinata-san-francisco',
 '/biz/apres-winter-lounge-east-palo-alto',
 '/biz/xiao-long-bao-kitchen-south-san-francisco',
 '/biz/benkyodo-co-san-francisco',
 '/biz/mission-pie-san-francisco',
 '/biz/neighbor-bakehouse-san-francisco',
 '/biz/ginto-izakaya-japonaise-san-francisco',
 '/biz/chile-pies-baking-co-san-francisco',
 '/biz/goc

In [296]:
# store to recall later
%store self_bookmarks

Stored 'self_bookmarks' (list)


In [521]:
my_bm_ids = [id.split('/biz/')[1] for id in self_bookmarks]
my_bm_ids

['three-twins-ice-cream-san-francisco-5',
 'ben-thai-cafe-san-francisco',
 'ijji-sushi-san-francisco',
 'koi-palace-daly-city',
 'robin-san-francisco',
 'maria-catita-restaurante-e-loja-regional-lisboa',
 'juns-beauty-salon-san-francisco',
 'bowld-acai-san-francisco-2',
 'china-bee-san-mateo',
 'bobos-san-francisco-14',
 'grand-lake-kitchen-oakland',
 'nabe-san-francisco-5',
 'dumpling-kitchen-san-francisco',
 'nojo-ramen-tavern-san-francisco-2',
 'el-milagro-los-angeles-2',
 'five-happiness-san-francisco',
 'tadich-grill-san-francisco',
 'mahalo-bowl-sunnyvale-2',
 'hinata-san-francisco',
 'apres-winter-lounge-east-palo-alto',
 'xiao-long-bao-kitchen-south-san-francisco',
 'benkyodo-co-san-francisco',
 'mission-pie-san-francisco',
 'neighbor-bakehouse-san-francisco',
 'ginto-izakaya-japonaise-san-francisco',
 'chile-pies-baking-co-san-francisco',
 'gochi-japanese-fusion-tapas-cupertino',
 'pineapples-san-francisco',
 'the-pots-san-francisco',
 'huge-tree-pastry-monterey-park',
 'lihol

In [297]:
len(self_bookmarks)

249

In [420]:
# create dictionary of users and their bookmarks
all_bookmarks = {i:[] for i in all_friends}

for i in all_friends:
    all_bookmarks[i] = get_bookmarks(i)

In [422]:
%store all_bookmarks

Stored 'all_bookmarks' (dict)


In [ ]:
all_bookmarks

In [532]:
def create_df_bookmarks(key):
    df = pd.DataFrame(all_bookmarks[key], columns=['url'])
    df['id'] = key
    return df

In [533]:
all_bm_df = [create_df_bookmarks(key) for key in all_bookmarks.keys()]
all_bm_df = pd.concat(all_bm_df)

In [539]:
all_bm_df

,url,id
0,NaN,T1ZDOuZxeBK-SaowZqYl_w
0,/biz/comoncy-scottsdale-4,7IV3JXTXufasWNSkDEdrcQ
1,/biz/padrecito-san-francisco,7IV3JXTXufasWNSkDEdrcQ
2,/biz/everett-and-jones-barbeque-oakland-4,7IV3JXTXufasWNSkDEdrcQ
3,/biz/merrimans-kapalua-kapalua,7IV3JXTXufasWNSkDEdrcQ
4,/biz/aa-roots-lahaina-5,7IV3JXTXufasWNSkDEdrcQ
5,/biz/sasaki-san-francisco-2,7IV3JXTXufasWNSkDEdrcQ
6,/biz/del-mar-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ
7,/biz/stonemill-matcha-san-francisco,7IV3JXTXufasWNSkDEdrcQ
8,/biz/the-lot-sf-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ


In [564]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [540]:
def split_biz_url(url):
    try:
        return url.split('/biz/')[1]       
    except:
        pass

In [558]:
len((all_bm_df[all_bm_df['business_id'].notnull()]))

20552

In [563]:
len(list(set(all_bm_df['business_id'])))

13820

In [551]:
all_bm_df['business_id'] = all_bm_df['url'].apply(lambda url: split_biz_url(url))
all_bm_df.reset_index()

,index,url,id,business_id
0,0,NaN,T1ZDOuZxeBK-SaowZqYl_w,None
1,0,/biz/comoncy-scottsdale-4,7IV3JXTXufasWNSkDEdrcQ,comoncy-scottsdale-4
2,1,/biz/padrecito-san-francisco,7IV3JXTXufasWNSkDEdrcQ,padrecito-san-francisco
3,2,/biz/everett-and-jones-barbeque-oakland-4,7IV3JXTXufasWNSkDEdrcQ,everett-and-jones-barbeque-oakland-4
4,3,/biz/merrimans-kapalua-kapalua,7IV3JXTXufasWNSkDEdrcQ,merrimans-kapalua-kapalua
5,4,/biz/aa-roots-lahaina-5,7IV3JXTXufasWNSkDEdrcQ,aa-roots-lahaina-5
6,5,/biz/sasaki-san-francisco-2,7IV3JXTXufasWNSkDEdrcQ,sasaki-san-francisco-2
7,6,/biz/del-mar-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ,del-mar-san-francisco-3
8,7,/biz/stonemill-matcha-san-francisco,7IV3JXTXufasWNSkDEdrcQ,stonemill-matcha-san-francisco
9,8,/biz/the-lot-sf-san-francisco-3,7IV3JXTXufasWNSkDEdrcQ,the-lot-sf-san-francisco-3


In [344]:
# get number of unique bookmarks from all friends

bm_vocab = [item for sl in all_bookmarks.values() for item in sl]
bm_set = list(set(bm_vocab))
len(bm_set)

10652

In [346]:
# create vectors for all friends and bookmarks - 1 for if they bookmarked and 0 if they did not
all_usr_vector = []

for k,v in all_bookmarks.items():
    usr_vector = []
    for bm in bm_set:
        if bm in v:
            usr_vector.extend([1])
        else:
            usr_vector.extend([0])
    all_usr_vector.append(usr_vector)

In [347]:
# create dataframe
bm_df = pd.DataFrame(all_usr_vector, columns=bm_set, index=all_bookmarks.keys())

bm_df

,nan,/biz/ben-thai-cafe-san-francisco,/biz/sushi-shibucho-costa-mesa,/biz/caffe-roma-coffee-roasting-san-francisco,/biz/pippin-vintage-jewelry-new-york,/biz/pok-pok-noi-portland,/biz/kiku-sushi-berkeley,/biz/carrera-cafe-los-angeles,/biz/handley-cellars-philo-2,/biz/coffee-nature-costa-mesa,...,/biz/lombard-street-san-francisco-3,/biz/polished-nest-san-francisco-2,/biz/swensens-ice-cream-san-francisco,/biz/cha-cafe-west-covina-2,/biz/shanghai-dumpling-shop-millbrae,/biz/chateau-boswell-st-helena,/biz/mckinney-farms-brentwood,/biz/la-boucherie-los-angeles-4,/biz/alta-new-york,/biz/vien-huong-restaurant-oakland
7IV3JXTXufasWNSkDEdrcQ,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T1ZDOuZxeBK-SaowZqYl_w,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oLh6PxTxdMAufjerdyZz5w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ipxRiIT-aL-M2nA-iaWOWA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e0zzk2kQHN2ghZHEGM-ytg,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ycztD9CWXLfDk4bPmA_tnw,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5RifcJP_Lf-MzojTHybBNw,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
NmdKCZlUVJGnqDxhXl2ofw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QjLiYeQLeMIqYu-ncmkUXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tdowJfZymyZNJymLmBfZeA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [348]:
# add my bookmarks
my_bm_comp = []
for i in bm_df.columns:
    if i in self_bookmarks:
        my_bm_comp.append(1)
    else:
        my_bm_comp.append(0)

mbm = pd.Series(my_bm_comp).to_frame(self_id).T
mbm.columns = bm_df.columns
mbm

,nan,/biz/ben-thai-cafe-san-francisco,/biz/sushi-shibucho-costa-mesa,/biz/caffe-roma-coffee-roasting-san-francisco,/biz/pippin-vintage-jewelry-new-york,/biz/pok-pok-noi-portland,/biz/kiku-sushi-berkeley,/biz/carrera-cafe-los-angeles,/biz/handley-cellars-philo-2,/biz/coffee-nature-costa-mesa,...,/biz/lombard-street-san-francisco-3,/biz/polished-nest-san-francisco-2,/biz/swensens-ice-cream-san-francisco,/biz/cha-cafe-west-covina-2,/biz/shanghai-dumpling-shop-millbrae,/biz/chateau-boswell-st-helena,/biz/mckinney-farms-brentwood,/biz/la-boucherie-los-angeles-4,/biz/alta-new-york,/biz/vien-huong-restaurant-oakland
ShHBKjuJbQAVBLs7DgA95A,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [349]:
bookmark_df = pd.concat([bm_df, mbm])
bookmark_df

,nan,/biz/ben-thai-cafe-san-francisco,/biz/sushi-shibucho-costa-mesa,/biz/caffe-roma-coffee-roasting-san-francisco,/biz/pippin-vintage-jewelry-new-york,/biz/pok-pok-noi-portland,/biz/kiku-sushi-berkeley,/biz/carrera-cafe-los-angeles,/biz/handley-cellars-philo-2,/biz/coffee-nature-costa-mesa,...,/biz/lombard-street-san-francisco-3,/biz/polished-nest-san-francisco-2,/biz/swensens-ice-cream-san-francisco,/biz/cha-cafe-west-covina-2,/biz/shanghai-dumpling-shop-millbrae,/biz/chateau-boswell-st-helena,/biz/mckinney-farms-brentwood,/biz/la-boucherie-los-angeles-4,/biz/alta-new-york,/biz/vien-huong-restaurant-oakland
7IV3JXTXufasWNSkDEdrcQ,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T1ZDOuZxeBK-SaowZqYl_w,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
oLh6PxTxdMAufjerdyZz5w,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
ipxRiIT-aL-M2nA-iaWOWA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
e0zzk2kQHN2ghZHEGM-ytg,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ycztD9CWXLfDk4bPmA_tnw,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5RifcJP_Lf-MzojTHybBNw,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
NmdKCZlUVJGnqDxhXl2ofw,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QjLiYeQLeMIqYu-ncmkUXg,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tdowJfZymyZNJymLmBfZeA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# create a mapping for index number of bookmarks dataframe and user ID to be used later
index_map = pd.DataFrame({'user_id':bookmark_df.index})
index_map.index = index_map.index +1
index_map

In [350]:
# checking to see if there is overlap
l1 = bookmark_df.loc[self_id,:][bookmark_df.loc[self_id,:]==1].index.values
l2 = self_bookmarks

a = set(l1)
b = set(l2)
print('There is overlap: '+ str(len(bm_set) > len(b.difference(a))))
print(str(len(b.difference(a))) + ' out of ' + str(len(self_bookmarks)) + ' of my bookmarks are not bookmarked by others')

There is overlap: True
70 out of 249 of my bookmarks are not bookmarked by others


## Calculate Similarity - Bookmarks

Now that I have created the dataframe and confirmed that there is overlap in my bookmarks and my friend's bookmarks, I can calculate the similarities between myself and my friends.



In [352]:
sim_score = {}
for i in bookmark_df.index:
    ss = pearsonr(bookmark_df.loc[self_id,:], bookmark_df.loc[i,:])
    sim_score.update({i: ss[0]})

bm_sim = pd.Series(sim_score).to_frame('bm_similarity')
bm_sim_sorted = bm_sim.sort_values('bm_similarity', ascending=False)
bm_sim_sorted

,bm_similarity
ShHBKjuJbQAVBLs7DgA95A,1.000000
Zw-0Lo01W0QdkHT60DYg4g,0.096375
UaKdT4twgZ4DguHJhT6vPw,0.083417
oLh6PxTxdMAufjerdyZz5w,0.080619
aqC54BcscB12jMCQcyga1g,0.078492
-OKmukwdCrHq6bkF2_gSwQ,0.077621
yDy1xHRcFG4LWU3rvlOxUQ,0.072269
NmdKCZlUVJGnqDxhXl2ofw,0.064376
MzC1_5kXxGw336fMYSrsdg,0.058330
7IV3JXTXufasWNSkDEdrcQ,0.046444


## Getting the data - Reviews

In a similar fashion, I scraped www.yelp.com for my own reviews and my friend's reviews. Once I have the data, I will calculate the similarity between myself and my friends

In [303]:
# scrape my own reviews
my_reviews = get_reviews(self_id)
my_reviews

{'/biz/ace-wasabi-rock-n-roll-sushi-san-francisco': 5,
 '/biz/adventure-cat-sailing-charters-san-francisco-2': 5,
 '/biz/aji-peruvian-cuisine-long-beach-3': 4,
 '/biz/appethaizing-portland': 4,
 '/biz/baohaus-new-york-2': 4,
 '/biz/be-fresh-la-jolla': 5,
 '/biz/beni-tora-los-angeles-3': 4,
 '/biz/cafe-21-san-diego-2': 5,
 '/biz/caffe-centro-san-francisco': 3,
 '/biz/champa-garden-san-francisco-2': 3,
 '/biz/cherry-blossom-bakery-san-francisco-2': 5,
 '/biz/china-stix-restaurant-santa-clara': 5,
 '/biz/chouchou-new-york-2': 3,
 '/biz/cupertino-iphone-repair-cupertino-5': 2,
 '/biz/delicious-food-corner-monterey-park': 2,
 '/biz/elmers-restaurant-portland-3': 3,
 '/biz/em-lash-and-brow-san-jose': 5,
 '/biz/farallon-san-francisco': 3,
 '/biz/fuji-sukiyaki-san-mateo': 3,
 '/biz/furlong-vision-correction-medical-center-san-jose': 5,
 '/biz/genwa-korean-bbq-beverly-hills': 5,
 '/biz/gogigo-korean-bbq-cupertino': 3,
 '/biz/hinata-san-francisco': 5,
 '/biz/ifixers-iphone-repair-san-jose-18': 5

In [304]:
%store my_reviews

Stored 'my_reviews' (dict)


In [3]:
%store -r

In [12]:
my_review_df = pd.DataFrame(list(my_reviews.items()), columns=['business', 'rating'])
my_review_df['id'] = self_id
my_review_df.head()

,business,rating,id
0,/biz/prubechu-san-francisco,5,ShHBKjuJbQAVBLs7DgA95A
1,/biz/em-lash-and-brow-san-jose,5,ShHBKjuJbQAVBLs7DgA95A
2,/biz/tora-ya-ramen-ville-de-qu%C3%A9bec,3,ShHBKjuJbQAVBLs7DgA95A
3,/biz/fuji-sukiyaki-san-mateo,3,ShHBKjuJbQAVBLs7DgA95A
4,/biz/ristorante-quattro-montr%C3%A9al-2,5,ShHBKjuJbQAVBLs7DgA95A


In [16]:
save_obj(my_reviews, 'my_reviews' )

In [13]:
my_review_df

,business,rating,id
0,/biz/prubechu-san-francisco,5,ShHBKjuJbQAVBLs7DgA95A
1,/biz/em-lash-and-brow-san-jose,5,ShHBKjuJbQAVBLs7DgA95A
2,/biz/tora-ya-ramen-ville-de-qu%C3%A9bec,3,ShHBKjuJbQAVBLs7DgA95A
3,/biz/fuji-sukiyaki-san-mateo,3,ShHBKjuJbQAVBLs7DgA95A
4,/biz/ristorante-quattro-montr%C3%A9al-2,5,ShHBKjuJbQAVBLs7DgA95A
5,/biz/milkweed-boston,4,ShHBKjuJbQAVBLs7DgA95A
6,/biz/lukes-lobster-downtown-crossing-boston,5,ShHBKjuJbQAVBLs7DgA95A
7,/biz/il-fornaio-santa-clara-2,3,ShHBKjuJbQAVBLs7DgA95A
8,/biz/larb-thai-food-and-tapas-el-cerrito,5,ShHBKjuJbQAVBLs7DgA95A
9,/biz/sprint-store-san-jose-8,1,ShHBKjuJbQAVBLs7DgA95A


In [305]:
# create dictionary of users and their reviews
all_reviews = {i:[] for i in source_friends}

for i in source_friends:
    all_reviews[i] = get_reviews(i)

In [ ]:
# create dictionary of all users and their reviews
all_reviews2 = {i:[] for i in all_friends}

for i in all_friends:
    all_reviews2[i] = get_reviews(i)

In [306]:
%store all_reviews

Stored 'all_reviews' (dict)


In [307]:
len(all_reviews)

48

In [368]:
all_reviews

{'-OKmukwdCrHq6bkF2_gSwQ': {'/biz/la-victoria-taqueria-san-jose-2': 2,
  '/biz/mancinis-sleepworld-colma': 5,
  '/biz/tavor-plumbing-san-bruno': 1},
 '5RifcJP_Lf-MzojTHybBNw': {'/biz/55-south-san-jose': 5,
  '/biz/99-chicken-santa-clara': 4,
  '/biz/a-1-performance-auto-repair-sunnyvale': 5,
  '/biz/adamsons-french-dip-sunnyvale': 3,
  '/biz/ai-noodle-cupertino': 3,
  '/biz/alexanders-steakhouse-san-francisco-7': 4,
  '/biz/anchor-bay-thai-kitchen-gualala': 4,
  '/biz/anikis-sushi-fremont': 5,
  '/biz/aquamaids-bingo-santa-clara-2': 5,
  '/biz/arco-sunnyvale': 5,
  '/biz/arlanda-express-stockholm': 4,
  '/biz/asian-pearl-seafood-restaurant-fremont': 3,
  '/biz/back-a-yard-san-jose': 5,
  '/biz/backyard-bayou-union-city-2': 3,
  '/biz/bagel-street-cafe-sunnyvale': 4,
  '/biz/base-camp-pizza-south-lake-tahoe': 5,
  '/biz/beauty-fusion-studio-sunnyvale-4': 2,
  '/biz/best-buy-san-carlos-san-carlos': 4,
  '/biz/bi-rite-creamery-san-francisco': 4,
  '/biz/big-e-cafe-san-jose': 5,
  '/biz/bj

In [ ]:
def create_df_reviews(key):
    df = pd.DataFrame(list(all_reviews[key].items()), columns=['business', 'rating'])
    df['id'] = key
    return df

In [408]:
pd.DataFrame(list(all_reviews['7IV3JXTXufasWNSkDEdrcQ'].items()), columns=['business', 'rating'])

,business,rating
0,/biz/trident-grill-tucson-2,5
1,/biz/sausage-deli-tucson,5
2,/biz/le-grenier-%C3%A0-pain-paris-8,5
3,/biz/dianes-bloody-mary-san-francisco,5
4,/biz/barcino-san-francisco-2,5
5,/biz/la-cave-gourmande-paris-2,5
6,/biz/cath%C3%A9drale-notre-dame-de-paris-paris,3
7,/biz/dumpling-kitchen-san-francisco,5
8,/biz/bare-bare-donostia,4
9,/biz/lombard-beauty-salon-san-francisco-2,4


In [376]:
test = [pd.DataFrame(list(all_reviews[key].items()), columns=['location', 'rating']) for key in all_reviews.keys()]

In [421]:
def create_df_reviews(key, friends_dict):
    df = pd.DataFrame(list(friends_dict[key].items()), columns=['business', 'rating'])
    df['id'] = key
    return df

In [411]:
test = [create_df_reviews(key, all_reviews) for key in all_reviews.keys()]

In [412]:
len(test)

48

In [413]:
df = pd.concat(test)

In [414]:
df

,business,rating,id
0,/biz/trident-grill-tucson-2,5,7IV3JXTXufasWNSkDEdrcQ
1,/biz/sausage-deli-tucson,5,7IV3JXTXufasWNSkDEdrcQ
2,/biz/le-grenier-%C3%A0-pain-paris-8,5,7IV3JXTXufasWNSkDEdrcQ
3,/biz/dianes-bloody-mary-san-francisco,5,7IV3JXTXufasWNSkDEdrcQ
4,/biz/barcino-san-francisco-2,5,7IV3JXTXufasWNSkDEdrcQ
5,/biz/la-cave-gourmande-paris-2,5,7IV3JXTXufasWNSkDEdrcQ
6,/biz/cath%C3%A9drale-notre-dame-de-paris-paris,3,7IV3JXTXufasWNSkDEdrcQ
7,/biz/dumpling-kitchen-san-francisco,5,7IV3JXTXufasWNSkDEdrcQ
8,/biz/bare-bare-donostia,4,7IV3JXTXufasWNSkDEdrcQ
9,/biz/lombard-beauty-salon-san-francisco-2,4,7IV3JXTXufasWNSkDEdrcQ


In [378]:
?pd.concat

In [308]:
rev_vocab = [item for sl in list(all_reviews.values()) for item in sl.keys()]
rev_set = list(set(rev_vocab))
len(rev_set)

4331

Creating the dataframe for the reviews is slightly different. Instead of having 0 or 1 for whether or not there was a review, I am inputting the actual rating the user gave for the review.

In [309]:
all_reviews_vector = []

for usr,rating in all_reviews.items():
    for key,val in rating.items():
        usr_vector = []
        for rev in rev_set:
            if rev in key:
                usr_vector.append(all_reviews[usr][key])
            else:
                usr_vector.append(np.nan)
    all_reviews_vector.append(usr_vector)


In [310]:
%store all_reviews_vector

Stored 'all_reviews_vector' (list)


In [311]:
rev_df = pd.DataFrame(all_reviews_vector, columns=rev_set, index=all_reviews.keys())

my_rev_comp = []
for i in rev_df.columns:
    if i in my_reviews:
        my_rev_comp.append(my_reviews[i])
    else:
        my_rev_comp.append(np.nan)

mrev = pd.Series(my_rev_comp).to_frame(self_id).T
mrev.columns = rev_df.columns

review_df = pd.concat([rev_df, mrev])

# normalize the ratings
review_df = review_df.apply(lambda row: normalize(row), axis=1)
review_df = review_df.fillna(0)
review_df.tail()

,/biz/iza-ramen-soma-san-francisco-2,/biz/ben-thai-cafe-san-francisco,/biz/coffeebar-truckee-5,/biz/adamsons-french-dip-sunnyvale,/biz/elite-week-sparkcycle-san-diego,/biz/san-diego-superior-court-traffic-division-san-diego,/biz/lake-miramar-san-diego,/biz/evergreen-panda-san-jose,/biz/bandit-san-francisco,/biz/luxe-buffet-carlsbad,...,/biz/le-boulanger-campbell-2,/biz/shanghai-dumpling-shop-millbrae,/biz/%E4%B9%9C%E5%98%9C%E9%9B%9E%E8%9B%8B%E4%BB%94-%E9%A6%99%E6%B8%AF,/biz/tatami-sushi-and-seafood-buffet-cupertino-2,/biz/health-hut-los-angeles,/biz/mr-bings-san-francisco,/biz/enterprise-rent-a-car-seatac-3,/biz/past%C3%A9is-de-bel%C3%A9m-lisboa-6,/biz/yokohama-redwood-city,/biz/lord-stanley-san-francisco
y8mTD1BMn8Y0nR1ca_sPLw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ilgZlytsBSXGa14RVnQg_Q,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FraRROMnu9ro_RACmHafDQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
j-IxKzypD5KYgPwB-8kb-A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ShHBKjuJbQAVBLs7DgA95A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [312]:
review_df.shape

(49, 4331)

In [313]:
l1 = review_df.iloc[-1,:][review_df.iloc[-1,:]==1].index.values
l2 = my_reviews

a = set(l1)
b = set(l2)
print('There is overlap: '+ str(len(rev_set) > len(b.difference(a))))
print(str(len(b.difference(a))) + ' out of ' + str(len(my_reviews)) + ' of my reviews are not reviewed by others')

There is overlap: True
67 out of 67 of my reviews are not reviewed by others


Based on the above, it appears that none of the friends on the first page, which includes Yelp Elite friends, did not review the same location as I did. If I had the time and computing power to scrape all 144 friends and get their reviews, there would hopefully be a higher likelihood of overlap. Nonetheless, we will continue with the recommender.

## Calculate Similarity - Reviews

In [354]:
rev_sim_score = {}
for i in review_df.index:
    ss = pearsonr(review_df.loc[self_id,:], review_df.loc[i,:])
    rev_sim_score.update({i: ss[0]})
    
rev_sim = pd.Series(rev_sim_score).to_frame('rev_similarity')
rev_sim_sorted = rev_sim.sort_values('rev_similarity', ascending=False)
rev_sim_sorted

/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


,rev_similarity
ShHBKjuJbQAVBLs7DgA95A,1.0
-OKmukwdCrHq6bkF2_gSwQ,NaN
5RifcJP_Lf-MzojTHybBNw,NaN
6sDNWHF_jKE9rIjbc5ZfOw,NaN
7IV3JXTXufasWNSkDEdrcQ,NaN
7XdR2OvFXBoOmQ05pALzmg,NaN
D7O2Euol8WGTGFWF6klNqg,NaN
FOdWCgLWE0b7QMcu3Q9MYQ,NaN
FraRROMnu9ro_RACmHafDQ,NaN
FxXqn273QfaKXOLyKHhJVw,NaN


## Combining Bookmarks & Reviews Similarities

Now that we have the similarities for both bookmarks and reviews, we can use both of them to determine the user that is most like us by simply adding the two similarities together. Based on my own experience and discussing with friends, some people will remove bookmarks once they have visited the establishment and then may leave a review if they are an active reviewer. As such, I believe that simply adding the two similarities to give us a final similarity is appropriate.

Note: In this case, we have no overlap in reviews so it is all NaN

In [355]:
both_sim = bm_sim.join(rev_sim)
both_sim['total'] = both_sim.sum(axis=1)
both_sim = both_sim.sort_values('total', ascending=False)
both_sim

,bm_similarity,rev_similarity,total
ShHBKjuJbQAVBLs7DgA95A,1.000000,1.0,2.000000
Zw-0Lo01W0QdkHT60DYg4g,0.096375,NaN,0.096375
UaKdT4twgZ4DguHJhT6vPw,0.083417,NaN,0.083417
oLh6PxTxdMAufjerdyZz5w,0.080619,NaN,0.080619
aqC54BcscB12jMCQcyga1g,0.078492,NaN,0.078492
-OKmukwdCrHq6bkF2_gSwQ,0.077621,NaN,0.077621
yDy1xHRcFG4LWU3rvlOxUQ,0.072269,NaN,0.072269
NmdKCZlUVJGnqDxhXl2ofw,0.064376,NaN,0.064376
MzC1_5kXxGw336fMYSrsdg,0.058330,NaN,0.058330
7IV3JXTXufasWNSkDEdrcQ,0.046444,NaN,0.046444


## Generate Recommendations

Now I'm ready to identify recommendations by concatenating the reviews and the bookmarks togethers, look at the top 3 similar users to myself, and then further look at places where at least 2 of the 3 users have also either bookmarked or reviewed.

In [367]:
# get top 3 similar users
top = both_sim.index[:4]

book_recs = bookmark_df.loc[top,:][bookmark_df.loc[top,:]==1].fillna(0).T
review_recs = review_df.loc[top,:][review_df.loc[top,:]==1].fillna(0).T
all_recs = pd.concat([book_recs, review_recs])

# subset dataframe to only have items that I have not already bookmarked
temp = all_recs[all_recs[self_id]==0].copy()
str_recs = temp.iloc[:,:-1].copy()

# subset further to get bookmarks that more than 2 friends have also bookmarked
str_recs[str_recs.sum(axis=1)>1]

,ShHBKjuJbQAVBLs7DgA95A,Zw-0Lo01W0QdkHT60DYg4g,UaKdT4twgZ4DguHJhT6vPw
/biz/good-mong-kok-bakery-san-francisco,0.0,1.0,1.0
/biz/pho-huynh-hiep-2-kevins-noodle-house-san-francisco,0.0,1.0,1.0
/biz/zero-zero-san-francisco,0.0,1.0,1.0
/biz/skool-san-francisco,0.0,1.0,1.0
/biz/thanh-long-san-francisco,0.0,1.0,1.0
/biz/pho-2000-san-francisco,0.0,1.0,1.0
/biz/hai-ky-noodles-san-francisco,0.0,1.0,1.0
/biz/j%C5%AB-ni-san-francisco-5,0.0,1.0,1.0
/biz/la-taqueria-san-francisco-2,0.0,1.0,1.0
/biz/petite-provence-portland-2,0.0,1.0,1.0


# Results

Based on the resulting recommendations, I can confirm that these indeed look like places that I would enjoy eating at. There are some places where I have already been to and can confirm that I enjoyed it, which brings me to the limitation of the data. I did not bookmark every place I went to so some of these recommendations are places that I have already visited and would be a useless recommendation. If the account was a regular check-in user, then we could remove places that had already been visited. Further improvements could be layering in Content-based filtering with the types of restaurants I generally bookmark (cuisine type/atmosphere/pricing/and other various characteristics featured on Yelp.com). 